In [12]:
import pandas as pd
import numpy as np
import itertools
from datetime import date,datetime

In [24]:
holidays={
2018: {1:[1,2,3,4,5,6,7,8,13,14,20,21,27,28],2:[3,4,10,11,17,18,22,23,24,25],3:[3,4,7,8,9,10,11,17,18,24,25,31],4:[1,7,8,14,15,21,22,28,29,30],5:[1,2,5,6,8,9,12,13,19,20,26,27],6:[2,3,9,10,11,12,16,17,23,24,30],7:[1,7,8,14,15,21,22,28,29],8:[4,5,11,12,18,19,25,26],9:[1,2,8,9,15,16,22,23,29,30],10:[6,7,13,14,20,21,27,28],11:[3,4,5,10,11,17,18,24,25],12:[1,2,8,9,15,16,22,23,29,30,31]},
2019: {1:[1,2,3,4,5,6,7,8,12,13,19,20,26,27],2:[2,3,9,10,16,17,22,23,24],3:[2,3,7,8,9,10,16,17,23,24,30,31],4:[6,7,13,14,20,21,27,28,30],5:[1,2,3,4,5,8,9,10,11,12,18,19,25,26],6:[1,2,8,9,11,12,15,16,22,23,29,30],7:[6,7,13,14,20,21,27,28],8:[3,4,10,11,17,18,24,25,31],9:[1,7,8,14,15,21,22,28,29],10:[5,6,12,13,19,20,26,27],11:[2,3,4,9,10,16,17,23,24,30],12:[1,7,8,14,15,21,22,28,29,31]},
2020: {1:[1,2,3,4,5,6,7,8,11,12,18,19,25,26],2:[1,2,8,9,15,16,22,23,24,29],3:[1,7,8,9,14,15,21,22,28,29],4:[4,5,11,12,18,19,25,26,30],5:[1,2,3,8,9,10,11,16,17,23,24,30,31],6:[6,7,11,12,13,14,20,21,27,28],7:[4,5,11,12,18,19,25,26],8:[1,2,8,9,15,16,22,23,29,30],9:[5,6,12,13,19,20,26,27],10:[3,4,10,11,17,18,24,25,31],11:[1,3,4,7,8,14,15,21,22,28,29],12:[5,6,12,13,19,20,26,27,31]},
2021: {1:[1,2,3,4,5,6,7,8,9,10,16,17,23,24,30,31],2:[6,7,13,14,20,21,22,23,27,28],3:[6,7,8,13,14,20,21,27,28],4:[3,4,10,11,17,18,24,25,30],5:[1,2,3,8,9,10,15,16,22,23,29,30],6:[5,6,11,12,13,14,19,20,26,27],7:[3,4,10,11,17,18,24,25,31],8:[1,7,8,14,15,21,22,28,29],9:[4,5,11,12,18,19,25,26],10:[2,3,9,10,16,17,23,24,30,31],11:[3,4,6,7,13,14,20,21,27,28],12:[4,5,11,12,18,19,25,26,31]}
}
holidays=pd.DataFrame(list(itertools.chain.from_iterable(list(itertools.chain.from_iterable([[[date(year,month,day) for day in days] for month,days in v.items()] for year,v in holidays.items()])))),columns=['date'])


In [5]:
def filetodf(filename,time_list=[]):
    # функция считывает файлы из директории и складывает данные в датафрейм
    #  входные данные: 
    #     filename - путь к файлам
    #     time_list - список полей с датой
    #  выходные данные: 
    #     pd.dataframe
    
    import glob
    import pandas as pd
    
    AllFiles = glob.glob(filename)
   
    list_ = [pd.DataFrame()]

    for file_ in AllFiles:
        columns=pd.read_csv(file_, sep=',', nrows=0).columns.str.lower().str.strip()
        df = pd.read_csv(file_,',',names=columns,parse_dates=time_list,dayfirst=True,skiprows=1)
        list_.append(df)
    
    return pd.concat(list_, axis = 0, ignore_index = True, sort=True)

In [6]:
df_sysMetrics = filetodf('input/GrpStat_OSGLOB*.dat')
df_grpMetrics = filetodf('input/GrpStat_RGALL*.dat')
df_directives = filetodf('input/GrpStat_Directives*.dat',time_list=['begin_time'])

df_sysMetrics.drop(['time'], axis=1,inplace=True)
df_grpMetrics.drop(['t_beg'], axis=1,inplace=True)

df_grpMetrics.rename(index=str, columns={"instance_number": "host", "usr_group": "consumer_group"}, inplace=True)
df_directives.rename(index=str, columns={"instance_number": "host", "begin_time": "time"}, inplace=True)

df_grpMetrics['consumer_group']=df_grpMetrics['consumer_group'].str.strip().str.lower()
df_directives['consumer_group']=df_directives['consumer_group'].str.strip().str.lower()
df_directives['plan_name']=df_directives['plan_name'].str.strip().str.lower()

df_grpMetrics.replace(' None', np.NaN, inplace=True)
df_directives.replace(' None', np.NaN, inplace=True)
df_sysMetrics.replace(' None', np.NaN, inplace=True)

df_directives.parallel_target_percentage=df_directives.parallel_target_percentage.astype(float)
df_directives.parallel_degree_limit_p1=df_directives.parallel_degree_limit_p1.astype(float)
df_directives.max_utilization_limit=df_directives.max_utilization_limit.astype(float)
df_grpMetrics.pga=df_grpMetrics.pga.astype(float)

In [40]:
#джойним все данные до уровня host, snap_id, consumer_group
df=df_grpMetrics.set_index(['host','snap_id','consumer_group']) \
.join(df_directives.set_index(['host','snap_id','consumer_group']))\
.reset_index()\
.merge(df_sysMetrics,on=['host','snap_id'])


In [ ]:
#df = df[df.consumer_group!='other_groups']
#df = df[df.consumer_group!='ods2exa_group']

In [41]:
# ресэмплинг до часа, чтобы сгладить ряд
# заполняем отсутствующие значения средними
# фильтруем неполные дни в начале и в конце

df = df.sort_values(['host','consumer_group','time'])
df = df.set_index(pd.DatetimeIndex(df['time']))
#-----------------------------
#df.loc[(df.consumer_group=='adw_unload_group')&(df.plan_name=='sandbox_plan'),'mgmt_p1']=26.
#-----------------------------

#df.max_utilization_limit.fillna(100,inplace=True)
#df.parallel_target_percentage.fillna(100,inplace=True)
#df.loc[:,'cpu_pct_maxul']=df.cpu/df.max_utilization_limit
#df.loc[:,'px_pct_parallel']=df.pxqry/(df.parallel_target_percentage*1400.0/100.0)
#df.loc[df.px_pct_parallel>2,'px_pct_parallel']=2.0
#df.loc[df.cpu_pct_maxul>2,'cpu_pct_maxul']=2.0
#df['CPU_min_treshold']=(df.mgmt_p1/df.max_utilization_limit)

columns=df.columns.drop(['host','consumer_group','time','snap_id','plan_name'],errors='ignore')

df=df.groupby(['host','consumer_group'])[columns].apply(lambda df: df.resample('H').mean().fillna(df.mean(axis=0))).reset_index(['host','consumer_group'])
df=df.groupby(['host','consumer_group',df.index.date]).filter(lambda x: (x.index.min().hour==0)&(x.index.max().hour==23)).reset_index()


In [23]:
holidays.head()

,time
0,2018-01-01
1,2018-01-02
2,2018-01-03
3,2018-01-04
4,2018-01-05


In [21]:
#datetime features

    #day_of_month
    #day_of_week
    #month_of_year
    #hour_of_day
    #is_holiday
    #quarter_of_year


df.loc[:,('day_of_months')]=df.time.dt.day
df.loc[:,('day_of_week')]=df.time.dt.dayofweek
df.loc[:,('month_of_year')]=df.time.dt.month
df.loc[:,('hour_of_day')]=df.time.dt.hour
df.loc[:,('is_weekend')]=df.time.dt.dayofweek.isin([5,6])*1
df.loc[:,('quarter_of_year')]=df.time.dt.quarter

df['date']=df.time.dt.strftime('%Y-%m-%d')
#df=df.loc[~df.date.isin(holidays.ds.values.astype('M8[D]'))].drop(columns=['date'])

df = pd.merge(df,holidays,on='date', how='left')

In [46]:
holidays.date.loc[100:]

100    2018-10-20
101    2018-10-21
102    2018-10-27
103    2018-10-28
104    2018-11-03
105    2018-11-04
106    2018-11-05
107    2018-11-10
108    2018-11-11
109    2018-11-17
110    2018-11-18
111    2018-11-24
112    2018-11-25
113    2018-12-01
114    2018-12-02
115    2018-12-08
116    2018-12-09
117    2018-12-15
118    2018-12-16
119    2018-12-22
120    2018-12-23
121    2018-12-29
122    2018-12-30
123    2018-12-31
124    2019-01-01
125    2019-01-02
126    2019-01-03
127    2019-01-04
128    2019-01-05
129    2019-01-06
          ...    
460    2021-09-26
461    2021-10-02
462    2021-10-03
463    2021-10-09
464    2021-10-10
465    2021-10-16
466    2021-10-17
467    2021-10-23
468    2021-10-24
469    2021-10-30
470    2021-10-31
471    2021-11-03
472    2021-11-04
473    2021-11-06
474    2021-11-07
475    2021-11-13
476    2021-11-14
477    2021-11-20
478    2021-11-21
479    2021-11-27
480    2021-11-28
481    2021-12-04
482    2021-12-05
483    2021-12-11
484    202

In [43]:
df['date']=df.time.dt.strftime('%Y-%m-%d')
df = pd.merge(df,holidays,on ='date',how='left')
df.head(50)

TypeError: object of type 'NoneType' has no len()

In [ ]:
#dummy features
#df=pd.concat([df,pd.get_dummies(df[['host']].astype(str),prefix='host')],axis=1,sort=False)
#df=pd.concat([df,pd.get_dummies(df[['consumer_group']],prefix='group')],axis=1,sort=False)

In [ ]:
# добавляем лаги исходного ряда в качестве признаков
for i in range(prediction_points, prediction_points+10):
    df["lag_{}".format(i)] = df.groupby(['host'])['sys_cpu'].shift(i)

In [ ]:
#df_week_shift=df[columns].shift(24*7)
#df_day_shift=df[columns].shift(24)
#for (host,group) in df.drop_duplicates(['host','consumer_group'])[['host','consumer_group']].values:
#    t=df[(df.host==host)&(df.consumer_group==group)][columns]
#    for row in t.index:
#        if t.loc[row].isnull().all():
#            row_day=df_day_shift.loc[row,columns]
#            row_week=df_week_shift.loc[row,columns]
#            if row_week.isnull().all():
#                if row_day.isnull().all():
#                    df.loc[row,columns]=t.loc[:,columns].mean(axis=0)
#                else:
#                    df.loc[row,columns]=row_day
#            else:
#                df.loc[row,columns]=row_week
#df.fillna(0,inplace=True)
            
            

In [6]:
df_ml.to_csv('clear_data/rm_features_ml.csv',';',index=False)
df_nn.to_csv('clear_data/rm_features_nn.csv',';',index=False)